In [ ]:
# Heart Failure Prediction

# TABLE OF CONTENTS

1-)Importing Libraries And Importing Data

2-)Data Review

3-)Exploratory Data Analysis

4-)Outlier Detection

5-)Preprocessing

6-)Build Model And Accurcy Value

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1-)Importing Libraries And Importing Data


In [ ]:
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from warnings import filterwarnings
from sklearn.metrics import confusion_matrix, accuracy_score
filterwarnings("ignore")


data =pd.read_csv("../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv")

# 2-) Data Review

In [ ]:
print(data.shape)

In [ ]:
print(data.head())



In [ ]:
print(data.info())

We haven't got null data.

In [ ]:
print(data.describe().T)

# 3-)Exploratory Data Analysis

In [ ]:
corr_matrix =data.corr()
sns.clustermap(corr_matrix,annot=True,fmt=".2f")
plt.show()

In [ ]:
fig, axs = plt.subplots(4, 3, figsize=(12, 12))

sns.histplot(data=data["age"], kde=True, color="skyblue", ax=axs[0, 0])
sns.histplot(data=data["anaemia"], kde=False, color="olive", ax=axs[0, 1])
sns.histplot(data=data["creatinine_phosphokinase"], kde=True, color="gold", ax=axs[0, 2])

sns.histplot(data=data["diabetes"], kde=False, color="skyblue", ax=axs[1, 0])
sns.histplot(data=data["ejection_fraction"], kde=True, color="olive", ax=axs[1, 1])
sns.histplot(data=data["high_blood_pressure"], kde=False, color="gold", ax=axs[1, 2])

sns.histplot(data=data["platelets"], kde=True, color="skyblue", ax=axs[2, 0])
sns.histplot(data=data["serum_creatinine"], kde=True, color="olive", ax=axs[2, 1])
sns.histplot(data=data["serum_sodium"], kde=True, color="gold", ax=axs[2, 2])

sns.histplot(data=data["sex"], kde=False, color="skyblue", ax=axs[3, 0])
sns.histplot(data=data["smoking"], kde=False, color="olive", ax=axs[3, 1])
sns.histplot(data=data["time"], kde=True, color="gold", ax=axs[3, 2])

plt.show()

In [ ]:
zero =0
ones =0
for i in range(len(data["sex"])):
    if data["sex"][i] == 0:
        zero+=1
    else :
        ones+=1

names = ['zero', 'one']
size = [zero,ones]
 
my_circle = plt.Circle( (0,0), 0.7, color='white')

plt.pie(size, labels=names, colors=['red','green'])
p = plt.gcf()
p.gca().add_artist(my_circle)

plt.show()

In [ ]:
zero =0
ones =0
for i in range(len(data["smoking"])):
    if data["smoking"][i] == 0:
        zero+=1
    else :
        ones+=1
        
names = ['zero', 'one']

size = [zero,ones]
my_circle = plt.Circle( (0,0), 0.7, color='white')

plt.pie(size, labels=names, colors=['red','green'])
p = plt.gcf()
p.gca().add_artist(my_circle)

plt.show()

In [ ]:
zero =0
ones =0
for i in range(len(data["high_blood_pressure"])):
    if data["high_blood_pressure"][i] == 0:
        zero+=1
    else :
        ones+=1
        
names = ['zero', 'one']
size = [zero,ones]
 
my_circle = plt.Circle( (0,0), 0.7, color='white')

plt.pie(size, labels=names, colors=['red','green'])
p = plt.gcf()
p.gca().add_artist(my_circle)

plt.show()

# Effect Of Data On Death Event

In [ ]:
dev =data.corrwith(data['DEATH_EVENT'])
print(dev)

The properties with the most death event effect are serum creatinine, time and ejection_fraction.

In [ ]:
data.plot(y='serum_creatinine')
data.plot(y='creatinine_phosphokinase')
data.plot(y='ejection_fraction')


# 4-)Outlier Detection (IQR Method)

We will find outlier values of serum creatinine, creatine phosphokinase, ejection fraction properties by IQR Method and assign the mean values.

**1-) CREATININE PHOSPHOKINASE**

In [ ]:
#Creatinine phosphokinase outlier 

cp_out=data["creatinine_phosphokinase"]
sns.boxplot(y=data["creatinine_phosphokinase"] )
plt.show()

In [ ]:
q1_cp_out=data["creatinine_phosphokinase"].quantile(0.25) #Creatine phosphokinase  0.25 th value
q3_cp_out =data["creatinine_phosphokinase"].quantile(0.75)#Creatine phosphokinase  0.75 th value

print(q1_cp_out)
print(q3_cp_out)

In [ ]:
igr_cp_out =q3_cp_out-q1_cp_out
low_lim_cp_out =q1_cp_out-1.5*igr_cp_out #lower limit
up_lim_cp_out =q3_cp_out+1.5*igr_cp_out #upper limit


print(igr_cp_out)
print(low_lim_cp_out)
print(up_lim_cp_out)

In [ ]:
cp_out_mean=581.839465
cp_out[((cp_out>(up_lim_cp_out)) | (cp_out<(low_lim_cp_out)))]=cp_out_mean


print(data["creatinine_phosphokinase"])
sns.boxplot(y=data["creatinine_phosphokinase"] )
plt.show()


**2-) SERUM CREATININE**

In [ ]:
sc_out=data["serum_creatinine"]
sns.boxplot(y=data["serum_creatinine"] )
plt.show()


In [ ]:
q1_sc_out=data["serum_creatinine"].quantile(0.25)#serum creatinine 0.25 th value
q3_sc_out =data["serum_creatinine"].quantile(0.75)#serum creatinine 0.75 th value

print(q1_sc_out)
print(q3_sc_out)

In [ ]:
igr_sc_out =q3_sc_out-q1_sc_out
low_lim_p_out =q1_sc_out-1.5*igr_sc_out #lower limit
up_lim_p_out =q3_sc_out+1.5*igr_sc_out #upper limit

print(low_lim_p_out)
print(up_lim_p_out)

In [ ]:
sc_out_mean=1.393880
sc_out[((sc_out>(up_lim_p_out)) | (sc_out<(low_lim_p_out)))]=sc_out_mean
print(data["serum_creatinine"])

sns.boxplot(y=data["serum_creatinine"] )
plt.show()
print(sc_out.mean())

**3-) EJECTION FRACTION**

In [ ]:
ef_out =data["ejection_fraction"]

sns.boxplot(y=data["ejection_fraction"] )
plt.show()

In [ ]:
q1_ef_out=data["ejection_fraction"].quantile(0.25) #ejection_fraction 0.25 th value
q3_ef_out =data["ejection_fraction"].quantile(0.65) #ejection_fraction 0.65 th value

print(q1_ef_out)
print(q3_ef_out)

In [ ]:
igr_ef_out =q3_ef_out-q1_ef_out
low_lim_ef_out =q1_ef_out-1.5*igr_ef_out#lower limit
up_lim_ef_out =q3_ef_out+1.5*igr_ef_out #upper limit


print(low_lim_ef_out)
print(up_lim_ef_out)

In [ ]:
ef_out_mean=38.083612
ef_out[((ef_out>(up_lim_ef_out)) | (ef_out<(low_lim_ef_out)))]=ef_out_mean

print(data["ejection_fraction"])

sns.boxplot(y=data["ejection_fraction"] )
plt.show()

# Split Data


In [ ]:
y=data["DEATH_EVENT"]
x=data.drop("DEATH_EVENT",axis=1)


x_train,x_test,y_train,y_test =train_test_split(x,y,test_size=0.2,random_state=1)

# 5-)Standardize

In [ ]:

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# 6-)Build Model And Accurcy Value

In [ ]:
## Model creation with Xgboost 


from xgboost import XGBClassifier

model =XGBClassifier().fit(x_train,y_train)
y_pred =model.predict(x_test)
ac = accuracy_score(y_test,y_pred)
con = confusion_matrix(y_test, y_pred)
accuracies = []
accuracies.append(ac)
print("Xgboost model accuary",ac)
print(con)
    

In [ ]:
## Model creation with RandomForestClassifier 

from sklearn.ensemble import RandomForestClassifier


model =RandomForestClassifier().fit(x_train,y_train)
y_pred =model.predict(x_test)
ac = accuracy_score(y_test,y_pred)
con = confusion_matrix(y_test, y_pred)
accuracies = []
accuracies.append(ac)
print("RandomForestClassifier model accuary",ac)
print(con)
    

In [ ]:
## Model creation with SVC

from sklearn.svm import SVC

model = SVC().fit(x_train, y_train)

y_pred = model.predict(x_test)
ac = accuracy_score(y_test,y_pred)
con = confusion_matrix(y_test, y_pred)
accuracies = []
accuracies.append(ac)
print("SVC model accuary",ac)
print(con)